In [1]:
import pandas as pd
import re
import os
import psycopg2 as sql
from datetime import datetime as dt

In [2]:
ad_path = "D:\Program Files (x86)\World of Warcraft\_classic_\Interface\AddOns\TradeSkillMaster_AppHelper\AppData.lua"

In [4]:
last_modified = os.path.getmtime(ad_path)
last_modified = dt.fromtimestamp(last_modified)
print(last_modified)

2019-10-10 13:59:57


In [55]:
def cleanAppData(path):
    data_dir = os.path.dirname(path)
    
    # Find all conflicted AppData files
    data_files = os.listdir(data_dir)
    appdata_list = []
    for file in data_files:
        if re.match('AppData', file) is not None:
            appdata_list.append(file)
    
    # Find the most recent AppData file
    recmod = ''
    recmod_time = dt.min
    for file in appdata_list:
        file_ts = dt.fromtimestamp(os.path.getmtime(data_dir + '\\' + file))
        if file_ts > recmod_time:
            recmod_time = file_ts
            recmod = file
    
    # Move all AppData files that are not the most recent
    for file in appdata_list:
        if file != recmod:
            os.remove(data_dir + '\\' + file)
    
    # Rename the most recent AppData file to AppData.lua
    os.rename(data_dir + '\\' + recmod, data_dir + '\\AppData.lua')
    
    return

cleanAppData(data_path)

['AppData.lua']

In [5]:
def loadState():
    # If AppData path is saved, read file. If not, set earliest possible date
    ad_path = ''
    try: 
        with open('bin/adp.bin', 'rb') as file:
            ad_path_byte = file.read()
        ad_path = ad_path_byte.decode()
    except FileNotFoundError as e:        
        while ad_path == '':
            print('Please copy and paste World of Warcraft folder path in the format "<path to wow folder>/World of Warcraft"')
            ad_path = input('Path: ')

            # If extra directories to path are added 
            try:
                ad_path = re.search('(.*?)World of Warcraft', ad_path).group()
            except NoneTypeError as e:
                ad_path = ''
                continue
                

            if not os.path.exists(ad_path):
                ad_path = ''
                print('Directory does not exist. Unable to locate World of Warcraft folder.')
                continue
                
            ad_path += '\\_classic_\\Interface\\AddOns\\TradeSkillMaster_AppHelper'
            if not os.path.exists(ad_path):
                ad_path = ''
                raise FileNotFoundError("""
                Unable to locate TradeSkillMaster_AppHelper.
                Please download TradeSkillMaster_AppHelper at https://www.tradeskillmaster.com/install
                """)
            
            ad_path += '\\AppData.lua'
            if not os.path.isfile(ad_path):
                ad_path = ''
                raise FileNotFoundError('Unable to access data. Please reinstall TradeSkillMaster_AppHelper.')
                
    
    try: 
        with open('bin/lfm.bin', 'rb') as file:
            lst_mod_byte = file.read()
        lst_mod = dt.strptime(lst_mod_byte.decode(), '%Y-%m-%d %H:%M:%S.%f')
    except FileNotFoundError as e:     
        lst_mod = dt.min
        
        
    return ad_path, lst_mod

ad_path, lst_mod = loadState()
print(lst_mod)
print(ad_path)

2019-10-08 14:54:43
/home/ne3ko93/Games/world-of-warcraft-classic/drive_c/Program Files (x86)/World of Warcraft/_classic_/Interface/AddOns/TradeSkillMaster_AppHelper/AppData.lua


In [5]:
def parseAppData(path):
    with open(path, "r") as file:
        script = file.readlines()

    # Data we need is in the first line of the Interface\AddOns\TradeSkillMaster_AppHelper\AppData.lua file
    data_string = script[0]

    # Extract scan time from script
    scan_time = re.search('downloadTime=[0-9]+', data_string).group()
    scan_time = int(re.search('[0-9]+',scan_time).group())
    scan_time = dt.fromtimestamp(scan_time)
    
    # Extract columns from script
    col = re.search('fields={(.*?)}', data_string).group()
    col = re.search('{(.*?)}', col).group()
    col = re.sub('[{}"]','', col)
    col = col.split(',')
    col.append('scanTime')

    # Extract data from script
    data = re.search('data={(.*?)}}', data_string).group()
    data = re.search('{{(.*?)}}', data).group()
    data = re.sub('[{}]{2}','', data)
    data = data.split('},{')

    # Append data to dataframe
    df = pd.DataFrame(columns=col)
    for d_string in data:
        row = d_string.split(',')
        row.append(scan_time)
        df.loc[len(df)] = row
    
    # Convert all columns to dtype int (except scan time)
    for c in col:
        if c != 'scanTime':
            df[c] = df[c].astype(int)
            
    return df

df = parseAppData(ad_path)
df.head()

,itemString,marketValue,minBuyout,historical,numAuctions,scanTime
0,7757,85949,43599,101622,4,2019-10-10 13:10:55
1,14835,47845,0,322371,0,2019-10-10 13:10:55
2,3355,511,825,501,21,2019-10-10 13:10:55
3,2836,641,248,460,81,2019-10-10 13:10:55
4,15967,1405631,0,986540,0,2019-10-10 13:10:55


In [6]:
def mergeNewData(new_data):
    # If you do not have any stored data, create new file
    try:
        old_data = pd.read_csv('data/auction_data.csv')
    except FileNotFoundError as e:
        new_data.to_csv('data/auction_data.csv', index=False)
        return
    
    
    # If parsed data is the most recent, append to old data and store
    recent_scan = max(pd.to_datetime(old_data.scanTime))
    if max(new_data.scanTime) > recent_scan:
        data = new_data.append(old_data)
        data.to_csv('data/auction_data.csv', index=False)
    
    return

mergeNewData(df)

In [47]:
def saveState(lst_mod, ad_path):
    # Convert date to binary
    lst_mod_byte = lst_mod.strftime('%Y-%m-%d %H:%M:%S.%f').encode('utf-8')
    with open('bin/lfm.bin', 'wb') as file:
        file.write(bytearray(lst_mod_byte))
    
    ad_path_byte = ad_path.encode('utf-8')
    with open('bin/adp.bin', 'wb') as file:
        file.write(bytearray(ad_path_byte))
    
    return

saveState(last_modified, data_path)

In [56]:
import time

time.sleep(1)

In [8]:
def insertNewData(df, db='postgres', usr='njalexander93', pw='B2BzvRA63XB6JzuT99k2', host='wowaucdb.cvw2xhoownbe.us-east-2.rds.amazonaws.com', port='5432'):
    print("Attempting to connect to host {0}".format(host))
    try:
        conn = sql.connect(dbname=db, user=usr, password=pw, host=host, port=port)
    except:
        print('Unable to reach host {0}'.format(host))
        return
    print("Connection Successful!")
    cur = conn.cursor()
    
    with open('sqlscripts/rec_scantime.sql') as file:
        script = file.read()
    cur.execute(script)
    max_time = cur.fetchall()[0]
    max_time = max_time[0]
    
    scan_time = max(df.scanTime.tolist())
    if scan_time >= max_time:
    
        col_string = ""
        for col in df.columns.tolist():
            col_string += col + ", "
        col_string = col_string[:-2]

        data_string = ""
        for i in range(len(df)):
            data_string += '('
            for j in range(len(df.columns)):
                if j == len(df.columns) - 1:
                    data_string += "to_timestamp('{0}','YYYY-MM-DD HH24:MI:SS')".format(df.iloc[i,j]) + "),\n"
                else: 
                    data_string += str(df.iloc[i,j]) + ', '
        data_string = data_string[:-2]


        with open('sqlscripts/insert_data.sql') as file:
            script = file.read().format(col_string, data_string)
        try:
#             cur.execute(script)
            print(script)
        except:
            print("Unable to insert new data.")
            return
    else:
        print("No new data to insert.")
        
    conn.close()
    return

insertNewData(df)

Attempting to connect to host wowaucdb.cvw2xhoownbe.us-east-2.rds.amazonaws.com
Connection Successful!
INSERT INTO auc_hist(itemString, marketValue, minBuyout, historical, numAuctions, scanTime)
VALUES
    (7757, 85949, 43599, 101622, 4, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(14835, 47845, 0, 322371, 0, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(3355, 511, 825, 501, 21, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(2836, 641, 248, 460, 81, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(15967, 1405631, 0, 986540, 0, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(5640, 464, 565, 1123, 7, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(13093, 37529, 0, 34197, 0, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(10328, 281795, 249999, 448846, 2, to_timestamp('2019-10-10 13:10:55','YYYY-MM-DD HH24:MI:SS')),
(14604, 13816, 14624, 14358, 3, to_timestamp('2019-10-10 13:10:55